In [0]:
pip install azure-storage-blob xlrd==1.2.0

Python interpreter will be restarted.
Collecting azure-storage-blob
 Downloading azure_storage_blob-12.8.1-py2.py3-none-any.whl (345 kB)
Collecting xlrd==1.2.0
 Downloading xlrd-1.2.0-py2.py3-none-any.whl (103 kB)
Collecting azure-core<2.0.0,>=1.10.0
 Downloading azure_core-1.17.0-py2.py3-none-any.whl (165 kB)
Collecting msrest>=0.6.18
 Downloading msrest-0.6.21-py2.py3-none-any.whl (85 kB)
Requirement already satisfied: cryptography>=2.1.4 in /databricks/python3/lib/python3.8/site-packages (from azure-storage-blob) (3.1.1)
Requirement already satisfied: requests>=2.18.4 in /databricks/python3/lib/python3.8/site-packages (from azure-core<2.0.0,>=1.10.0->azure-storage-blob) (2.24.0)
Requirement already satisfied: six>=1.11.0 in /databricks/python3/lib/python3.8/site-packages (from azure-core<2.0.0,>=1.10.0->azure-storage-blob) (1.15.0)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from msrest>=0.6.18->azure-storage-blob) (2021.5.30)
Collecting requests-oauthlib>=0.5.0
 Downloading requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
Collecting isodate>=0.6.0
 Downloading isodate-0.6.0-py2.py3-none-any.whl (45 kB)
Requirement already satisfied: cffi!=1.11.3,>=1.8 in /databricks/python3/lib/python3.8/site-packages (from cryptography>=2.1.4->azure-storage-blob) (1.14.3)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core<2.0.0,>=1.10.0->azure-storage-blob) (1.25.11)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core<2.0.0,>=1.10.0->azure-storage-blob) (2.10)
Requirement already satisfied: chardet<4,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core<2.0.0,>=1.10.0->azure-storage-blob) (3.0.4)
Collecting oauthlib>=3.0.0
 Downloading oauthlib-3.1.1-py2.py3-none-any.whl (146 kB)
Requirement already satisfied: pycparser in /databricks/python3/lib/python3.8/site-packages (from cffi!=1.11.3,>=1.8->cryptography>=2.1.4->azure-storage-blob) (2.20)
Installing collected packages: azure-core, oauthlib, requests-oauthlib, isodate, msrest, azure-storage-blob, xlrd
Successfully installed azure-core-1.17.0 azure-storage-blob-12.8.1 isodate-0.6.0 msrest-0.6.21 oauthlib-3.1.1 requests-oauthlib-1.3.0 xlrd-1.2.0
WARNING: You are using pip version 20.2.4; however, version 21.2.4 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-bc88cb9d-a363-473b-9606-f28f710b8778/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
from pyspark.sql import SparkSession
from azure.storage.blob import BlobServiceClient
from pyspark.sql.types import StructField, StructType, StringType
import pandas as pd
import os

In [0]:
# start spark session
spark = SparkSession.builder.appName("app").getOrCreate()
spark.conf.set("fs.azure.account.key.cahospitalstorage.blob.core.windows.net", \
    "w3EYeDH//L+J0uscsn3BYDatbFCcB7CUXK4MfOSNmwc16v3WAQ7EPcw2IpchUrelLVuowjfugmv99e0Os8J5Sw==")

In [0]:
# connect to blob storage and download excel file
container = "insurance"
blob_location = "raw_data/CAInsurances.xlsx"

blob_service_client = BlobServiceClient.from_connection_string("DefaultEndpointsProtocol=https;AccountName=cahospitalstorage;AccountKey=w3EYeDH//L+J0uscsn3BYDatbFCcB7CUXK4MfOSNmwc16v3WAQ7EPcw2IpchUrelLVuowjfugmv99e0Os8J5Sw==;EndpointSuffix=core.windows.net")
container_client = blob_service_client.get_container_client(container)
blob_data = container_client.get_blob_client(blob_location)

with open("temp.xlsx", "wb") as temp:
  data = blob_data.download_blob()
  data.readinto(temp)

In [0]:
# read csv file
columns = ['CompanyName', 'CommonAlias', 'PhoneNumber', 'Website']
pdf = pd.read_excel("temp.xlsx", skiprows=[0], names=columns)
df = spark.createDataFrame(pdf)

In [0]:
# Export df_final to blob storage
output = "clean_data_insurance"

df.write.option("header", "true").mode("overwrite").parquet(output)
for file in os.listdir("/dbfs/" + output):
  try:
    with open("/dbfs/%s/%s" % (output, file), "rb") as data:
        container_client.upload_blob(name="%s/%s" % (output, file), data=data, overwrite=True)
  except Exception as e:
    print(e)

In [0]:
dbutils.notebook.exit("%s/%s" % (container, output))

insurance/clean_data_insurance